In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lightfm import LightFM
from lightfm.evaluation import precision_at_k,auc_score,reciprocal_rank
import scipy
import time
import math
from lightfm.data import Dataset
import sys
sys.path.append("/Users/noraxu/MSiA-Google/MSiABlog/")
import Rec_fx as rf

In [26]:
# read in data
data_users = pd.read_csv('/Users/noraxu/MSiA-Google/MSiABlog/users_toronto.csv',index_col=0)
data_business = pd.read_csv('/Users/noraxu/MSiA-Google/MSiABlog/business_Nora.csv',index_col=0)
#data_review = pd.read_csv('/Users/noraxu/MSiA-Google/MSiABlog/reviews_toronto.csv',index_col=0)
data_review = pd.read_csv('/Users/noraxu/MSiA-Google/MSiABlog/reviews_cleaned.csv',index_col=0)

In [27]:
data_review.cool = pd.Series([math.log(x+1) for x in data_review.cool])
data_review.useful = pd.Series([math.log(x+1) for x in data_review.useful])
data_review.funny = pd.Series([math.log(x+1) for x in data_review.funny])

In [28]:
data_business.iloc[:,24:].describe()

,Coffee & Tea,Nightlife,Bars,Specialty Food,Sandwiches,Breakfast & Brunch,Canadian (New),Cafes,Chinese,Italian,...,Halal,Gluten-Free,Food Delivery Services,Wine Bars,Delis,Health Markets,Tea Rooms,Sports Bars,Gastropubs,Tapas/Small Plates
count,10093.000000,10093.000000,10093.000000,10093.000000,10093.000000,10093.000000,10093.000000,10093.000000,10093.000000,10093.000000,...,10093.000000,10093.000000,10093.000000,10093.000000,10093.000000,10093.000000,10093.000000,10093.000000,10093.000000,10093.000000
mean,1.150003,0.634750,0.629464,0.833237,0.936205,0.858503,0.806546,0.860503,1.292694,1.164600,...,0.870183,0.739202,0.847934,0.563895,0.889617,0.755071,0.970510,0.572519,0.827071,0.736900
std,3.266034,1.849299,1.871181,3.056503,3.734560,3.510237,3.297011,3.564660,5.291207,4.951080,...,7.464651,6.466833,7.853991,5.095669,8.485234,7.102677,9.151189,5.279457,8.648734,7.930511
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,14.256579,16.783993,11.702970,24.348315,27.663830,27.861429,28.102305,28.680882,29.912577,30.908082,...,130.020000,90.923077,140.309353,60.009231,152.367188,102.377953,154.785714,78.641129,157.282258,189.349515


In [29]:
#model
dataset = Dataset()
dataset.fit(data_review.user_id,data_review.business_id)
type(dataset)
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))


Num users: 92381, num_items 10093.


In [30]:
# fit item and user features. 
dataset.fit_partial(items=data_business.business_id,
                    item_features=['stars'])
# dataset.fit_partial(items=data_business.business_id,
# #                     item_features=data_business.longitude)
# # dataset.fit_partial(items=data_business.business_id,
# #                     item_features=data_business.latitude)
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business.review_count)
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Coffee & Tea'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Nightlife'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Bars'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Specialty Food'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Sandwiches'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Breakfast & Brunch'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Canadian (New)'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Cafes'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Chinese'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Italian'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Bakeries'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Pizza'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Japanese'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Desserts'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Fast Food'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Burgers'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['American (Traditional)'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Sushi Bars'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Event Planning & Services'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Grocery'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Nightlife'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Indian'])
# dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business['Middle Eastern'])

# 'Mediterranean', 'Asian Fusion',
 #      'Ice Cream & Frozen Yogurt', 'Thai', 'Mexican', 'Pubs'
# # fit tags
# start_idx = 24
# for i in range(start_idx,len(data_business.columns)):
#     dataset.fit_partial(items=data_business.business_id,
#                     item_features=data_business[data_business.columns[i]])
   
    
tar_cols = [x for x in data_business.columns[24:]]

dataset.fit_partial(items = data_business.business_id,
                   item_features = tar_cols)                               
                                                                                                
                                                
# dataset.fit_partial(users=data_users.user_id,
#                     user_features=data_users.review_count)
# dataset.fit_partial(users=data_users.user_id,
#                     user_features=data_users.is_elite)
# dataset.fit_partial(users=data_users.user_id,
#                     user_features=data_users.useful)
# dataset.fit_partial(users=data_users.user_id,
#                     user_features=data_users.year)




In [31]:
user_cols = [x for x in data_users.columns[[18,19,22,23]]]
dataset.fit_partial(users=data_users.user_id,
                    user_features = user_cols)

In [32]:
#target_cols = ['stars','longitude','latitude','review_count']+[x for x in data_business.columns[24:]]
# target_cols = ['stars','review_count','Coffee & Tea', 'Nightlife',
#        'Bars', 'Specialty Food', 'Sandwiches', 'Breakfast & Brunch',
#        'Canadian (New)', 'Cafes', 'Chinese', 'Italian', 'Bakeries', 'Pizza',
#        'Japanese', 'Desserts', 'Fast Food', 'Burgers',
#        'American (Traditional)', 'Sushi Bars', 'Event Planning & Services',
#        'Grocery', 'Indian', 'Middle Eastern', 'Mediterranean', 'Asian Fusion',
#        'Ice Cream & Frozen Yogurt', 'Thai', 'Mexican', 'Pubs']#,'review_count']+[x for x in data_business.columns[24:]]

# print(target_cols)

# item_features = dataset.build_item_features([(x['business_id'], 
#                                               [y for y in x[target_cols]])
#                                               for index,x in data_business.iterrows()],normalize = True)
# user_features = dataset.build_user_features([(x['user_id'],
#                                              [x['review_count'],x['is_elite'],x['useful'],x['year']])
#                                            for index, x in data_users.iterrows()], normalize =False)

# print(repr(item_features))
# print(item_features.shape)

# print(repr(user_features))
# print(user_features.shape)

#build interaction
(interactions, weights) = dataset.build_interactions([(x['user_id'],
                                                       x['business_id'],
                                                       x['stars']) for index,x in data_review.iterrows()])

In [33]:

#build interaction
def build_dict(df,tar_cols):
    rst = {}
    for col in tar_cols:
        if(df[col]>1):
            rst[col]=math.log(df[col])
        else:
            rst[col] = df[col]
    return rst

item_features = dataset.build_item_features(((x['business_id'], 
                                              {'stars':x['stars'],**build_dict(x,tar_cols)})
                                              for index,x in data_business.iterrows()))

user_features = dataset.build_user_features([(x['user_id'],
                                             {'review_count':x['review_count'],'is_elite':x['is_elite'],
                                              'useful':x['useful'],'year':x['year']})
                                           for index, x in data_users.iterrows()])

In [34]:
#train-test split

from lightfm.cross_validation import random_train_test_split
train,test=random_train_test_split(interactions,test_percentage=0.2,random_state=np.random.RandomState(123))

print('The dataset has %s users and %s items, '
      'with %s interactions in the test and %s interactions in the training set.'
      % (train.shape[0], train.shape[1], test.getnnz(), train.getnnz()))

The dataset has 92381 users and 10093 items, with 82127 interactions in the test and 328507 interactions in the training set.


In [14]:
#model construction

from lightfm import LightFM

# Set the number of threads; you can increase this
# if you have more physical cores available.
NUM_THREADS = 2
NUM_COMPONENTS = 90    
NUM_EPOCHS = 30
ITEM_ALPHA = 1e-6
learning_rate=0.03

# Let's fit a WARP model: these generally have the best performance.
model = LightFM(loss='logistic',random_state=123,
                item_alpha=ITEM_ALPHA,
               no_components=NUM_COMPONENTS,
               learning_rate=learning_rate)

# Run 3 epochs and time it.
%time model = model.fit(train,epochs=NUM_EPOCHS,num_threads=NUM_THREADS)


# Import the evaluation routines
from lightfm.evaluation import auc_score

# Compute and print the AUC score
train_auc = auc_score(model, train, num_threads=NUM_THREADS).mean()
print('Collaborative filtering train AUC: %s' % train_auc)#logistic: 0.83, warp: 0.997

# We pass in the train interactions to exclude them from predictions.
# This is to simulate a recommender system where we do not
# re-recommend things the user has already interacted with in the train
# set.


test_auc = auc_score(model, test,num_threads=NUM_THREADS).mean()
print('Collaborative filtering test AUC: %s' % test_auc)#test: 0.828, warp:0.825


CPU times: user 23.1 s, sys: 138 ms, total: 23.2 s
Wall time: 23.5 s
Collaborative filtering train AUC: 0.82998383
Collaborative filtering test AUC: 0.8219651


In [ ]:
# Set the number of threads; you can increase this
# if you have more physical cores available.
NUM_THREADS = 2
NUM_COMPONENTS = 90    
NUM_EPOCHS = 30
ITEM_ALPHA = 1e-6
learning_rate=0.03

# Let's fit a WARP model: these generally have the best performance.
model = LightFM(loss='warp',random_state=123,
                item_alpha=ITEM_ALPHA,
               no_components=NUM_COMPONENTS,
               learning_rate=learning_rate)

# Run 3 epochs and time it.
%time model = model.fit(train,epochs=NUM_EPOCHS,num_threads=NUM_THREADS)


# Import the evaluation routines
from lightfm.evaluation import auc_score

# Compute and print the AUC score
train_auc = auc_score(model, train, num_threads=NUM_THREADS).mean()
print('Collaborative filtering train AUC: %s' % train_auc)#logistic: 0.83, warp: 0.997

# We pass in the train interactions to exclude them from predictions.
# This is to simulate a recommender system where we do not
# re-recommend things the user has already interacted with in the train
# set.


test_auc = auc_score(model, test,num_threads=NUM_THREADS).mean()
print('Collaborative filtering test AUC: %s' % test_auc)#test: 0.828, warp:0.825

In [ ]:
#Now include item features
model.ii = LightFM(loss='logistic',random_state=123,
                item_alpha=ITEM_ALPHA,
               no_components=NUM_COMPONENTS)

# Run 3 epochs and time it.
%time model.ii = model.ii.fit(train,item_features=item_features,epochs=NUM_EPOCHS,num_threads=NUM_THREADS)


# Import the evaluation routines
from lightfm.evaluation import auc_score

# Compute and print the AUC score
train_auc = auc_score(model.ii, train,item_features=item_features, num_threads=NUM_THREADS).mean()
print('Hybrid train AUC: %s' % train_auc)#0.9545

# We pass in the train interactions to exclude them from predictions.
# This is to simulate a recommender system where we do not
# re-recommend things the user has already interacted with in the train
# set.


test_auc = auc_score(model.ii, test,item_features=item_features,num_threads=NUM_THREADS).mean()
print('Hybrid test AUC: %s' % test_auc)#0.81,#0.803

In [15]:
#Now include item features
model.ii = LightFM(loss='warp',random_state=123,
                item_alpha=ITEM_ALPHA,
               no_components=NUM_COMPONENTS)

# Run 3 epochs and time it.
%time model.ii = model.ii.fit(train,item_features=item_features,epochs=NUM_EPOCHS,num_threads=NUM_THREADS)


# Import the evaluation routines
from lightfm.evaluation import auc_score

# Compute and print the AUC score
train_auc = auc_score(model.ii, train,item_features=item_features, num_threads=NUM_THREADS).mean()
print(' Hybrid train AUC: %s' % train_auc)#0.9545

# We pass in the train interactions to exclude them from predictions.
# This is to simulate a recommender system where we do not
# re-recommend things the user has already interacted with in the train
# set.


test_auc = auc_score(model.ii, test,item_features=item_features,num_threads=NUM_THREADS).mean()
print(' Hybrid test AUC: %s' % test_auc)#0.81,#0.803

CPU times: user 9min 16s, sys: 1.49 s, total: 9min 18s
Wall time: 9min 22s
 Hybrid train AUC: 0.99385494
 Hybrid test AUC: 0.7357032


In [ ]:
#Combine user_feature
model.iii = LightFM(loss='logistic',
                item_alpha=ITEM_ALPHA,
               no_components=NUM_COMPONENTS)

# Run 3 epochs and time it.
%time model.iii = model.iii.fit(train,user_features=user_features,item_features=item_features,epochs=NUM_EPOCHS,num_threads=NUM_THREADS)


# Import the evaluation routines
from lightfm.evaluation import auc_score

# Compute and print the AUC score
train_auc = auc_score(model.iii, train,user_features=user_features,item_features=item_features, num_threads=NUM_THREADS).mean()
print(' Hybrid train AUC: %s' % train_auc)#0.945

# We pass in the train interactions to exclude them from predictions.
# This is to simulate a recommender system where we do not
# re-recommend things the user has already interacted with in the train
# set.


test_auc = auc_score(model.iii, test,user_features=user_features,item_features=item_features,num_threads=NUM_THREADS).mean()
print('Hybrid test AUC: %s' % test_auc)#0.789,#0.784


In [35]:
#Combine user_feature
model.iii = LightFM(loss='warp',
                item_alpha=ITEM_ALPHA,
               no_components=NUM_COMPONENTS)

# Run 3 epochs and time it.
%time model.iii = model.iii.fit(train,user_features=user_features,item_features=item_features,epochs=NUM_EPOCHS,num_threads=NUM_THREADS)


# Import the evaluation routines
from lightfm.evaluation import auc_score

# Compute and print the AUC score
train_auc = auc_score(model.iii, train,user_features=user_features,item_features=item_features, num_threads=NUM_THREADS).mean()
print('Hybrid train AUC: %s' % train_auc)#0.945

# We pass in the train interactions to exclude them from predictions.
# This is to simulate a recommender system where we do not
# re-recommend things the user has already interacted with in the train
# set.


test_auc = auc_score(model.iii, test,user_features=user_features,item_features=item_features,num_threads=NUM_THREADS).mean()
print('Hybrid test AUC: %s' % test_auc)#0.789,#0.784, #0.839

CPU times: user 11min 25s, sys: 955 ms, total: 11min 26s
Wall time: 11min 28s
Hybrid train AUC: 0.83197063
Hybrid test AUC: 0.8234972
